### 学会使用tensorflow hub构建模型，用IMDB数据集训练

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers

print("Tensorflow Version: ", tf.__version__)
print("Eager Mode:", tf.executing_eagerly())
print("Hub Version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")
print("Visible device", tf.config.experimental.get_visible_devices())

Tensorflow Version:  2.0.0
Eager Mode: True
Hub Version:  0.7.0
GPU is available
Visible device [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


#### 1. 下载IMDB数据集

In [3]:
train_validation_split = tfds.Split.TRAIN.subsplit([6,4])
(train_data, validation_data), test_data = tfds.load(name='imdb_reviews',
                                                    split=(train_validation_split, tfds.Split.TEST),
                                                    as_supervised=True)


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=429, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

#### 2.1 创建一个使用Tensorflow hub模型嵌入语句的keras层

In [7]:
embedding = "https://hub.tensorflow.google.cn/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                          dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=962, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

#### 2.2 构建完整模型

In [8]:
model = keras.Sequential()
model.add(hub_layer)
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


#### 3. 模型配置超参数

In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### 4. 开始训练

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 96ms/step - loss: 1.5059 - accuracy: 0.5014 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 2s 66ms/step - loss: 0.7546 - accuracy: 0.5041 - val_loss: 0.7147 - val_accuracy: 0.5322
Epoch 3/20
30/30 [==============================] - 2s 66ms/step - loss: 0.6883 - accuracy: 0.5775 - val_loss: 0.6687 - val_accuracy: 0.5946
Epoch 4/20
30/30 [==============================] - 2s 67ms/step - loss: 0.6468 - accuracy: 0.6334 - val_loss: 0.6338 - val_accuracy: 0.6492
Epoch 5/20
30/30 [==============================] - 2s 66ms/step - loss: 0.6113 - accuracy: 0.6770 - val_loss: 0.6030 - val_accuracy: 0.6875
Epoch 6/20
30/30 [==============================] - 2s 67ms/step - loss: 0.5807 - accuracy: 0.7115 - val_loss: 0.5708 - val_accuracy: 0.7199
Epoch 7/20
30/30 [==============================] - 2s 65ms/step - loss: 0.5429 - accuracy: 0.7471 - val_loss: 0.5378 - val_accuracy: 0.7522
Epoch

#### 5.评估模型

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 1s - loss: 0.3174 - accuracy: 0.8662
loss: 0.317
accuracy: 0.866
